# Validation of far field evaluation

In [ ]:
%env OMP_NUM_THREADS=1
import firedrake as fd
import numpy as np
import matplotlib.pyplot as plt
from scattering import solve, far_field, far_field_vol,\
    plot_mesh, plot_field, plot_far_field, plot_far_field_vol
from mesh import generate_mesh

In [ ]:
# sound speed
c = 340
# number of cells across PML
N = 2

## Circle-shaped obstacle

In [ ]:
# shape of obstacle
shape = "circle"
# shift obstacle to break symmetry
shift = (0.1, 0.1)
# set up PML
a0 = b0 = 2.0
a1 = b1 = 2.25
h0 = (a1 - a0) / N
# set up cut-off function for far field
R0 = 1.2
R1 = 1.9
if R0 is not None and R1 is not None:
    annular = True
else:
    annular = False

# generate mesh
cached_mesh = False
if cached_mesh:
    mesh = fd.Mesh(shape + str(0) + ".msh")
else:
    mesh = generate_mesh(a0, a1, b0, b1, shape, N, R0=R0, R1=R1, shift=shift)

In [ ]:
# visualize mesh
plot_mesh(mesh)

In [ ]:
# wavenumber of incident wave
k = 5
# direction of incident wave
d = fd.Constant([1., 0.])

# Dirichlet boundary condition for sound-soft obstacle
x = fd.SpatialCoordinate(mesh)
kdx = k * fd.dot(d, x)
inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
g = -inc

# compute approximate solution
uh = solve(mesh, k, a0, a1, b0, b1, g, annular=annular)

In [ ]:
# visualize scattered field
plot_field(uh, a0, a1, b0, b1)

In [ ]:
# visualize total field
u = fd.interpolate(uh + inc, uh.function_space())
plot_field(u, a0, a1, b0, b1)

In [ ]:
# plot far field pattern evaluated by boundary-based formula
# use boundary of obstacle
plot_far_field(k, uh)

In [ ]:
# plot far field pattern evaluated by boundary-based formula
# use arbitrary closed surface enclosing obstacle
plot_far_field(k, uh, boundary=4)

In [ ]:
# plot far field pattern evaluated by volume-based formula
plot_far_field_vol(k, uh, R0, R1)

In [ ]:
# verify reciprocity relation [1, Theorem 3.23]
cached_mesh = False
if cached_mesh:
    mesh = fd.Mesh(shape + str(3) + ".msh")
else:
    mesh = generate_mesh(a0, a1, b0, b1, shape, N, level=3, R0=R0, R1=R1)
k = 5
d = fd.Constant([1., 0.])
x = fd.SpatialCoordinate(mesh)
kdx = k * fd.dot(d, x)
inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
g = -inc
u1 = solve(mesh, k, a0, a1, b0, b1, g, annular=annular)

n = 20
theta = 2 * np.pi / n * np.arange(n)
ratio = []
for t in list(theta):
    x_hat = fd.Constant((np.cos(t), np.sin(t)))
    u_inf1_re, u_inf1_im = far_field_vol(k, u1, x_hat, R0, R1)
    
    x = fd.SpatialCoordinate(mesh)
    kdx = k * fd.dot(-x_hat, x)
    inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
    g = -inc
    u2 = solve(mesh, k, a0, a1, b0, b1, g, annular=annular)
    u_inf2_re, u_inf2_im = far_field_vol(k, u2, -d, R0, R1)

    ratio.append((u_inf2_re / u_inf1_re, u_inf2_im / u_inf1_im))
theta = np.append(theta, 0)
ratio = np.array(ratio + [ratio[0]])

fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={'projection': 'polar'},
                               constrained_layout=True)
ax1.plot(theta, ratio[:, 0])
ax1.set_title("Real part")
ax1.set_rlabel_position(90)
ax1.grid(True)
ax2.plot(theta, ratio[:, 1])
ax2.set_title("Imaginary part")
ax2.set_rlabel_position(90)
ax2.grid(True)
fig.suptitle("Reciprocity relation")

In [ ]:
# verify translation property [1, (5.3)]
a0 = b0 = 2.0
a1 = b1 = 2.25
h0 = (a1 - a0) / N
R0 = 1.2
R1 = 1.9

shift0 = (0.1, 0.1)
shift1 = (0.1, -0.1)
h = np.array([shift1[0] - shift0[0], shift1[1] - shift0[1]])

cached_mesh = False
if cached_mesh:
    mesh0 = fd.Mesh(shape + str(3) + "_0.msh")
    mesh1 = fd.Mesh(shape + str(3) + "_1.msh")
else:
    mesh0 = generate_mesh(a0, a1, b0, b1, shape, N, level=3, R0=R0, R1=R1, shift=shift0, suffix="_0")
    mesh1 = generate_mesh(a0, a1, b0, b1, shape, N, level=3, R0=R0, R1=R1, shift=shift1, suffix="_1")
k = 5
d = [1., 0.]
d_np = np.array(d)
d = fd.Constant(d)

x = fd.SpatialCoordinate(mesh0)
kdx = k * fd.dot(d, x)
inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
g = -inc
u0 = solve(mesh0, k, a0, a1, b0, b1, g, annular=annular)
plot_field(u0, a0, a1, b0, b1)
plot_far_field_vol(k, u0, R0, R1)

x = fd.SpatialCoordinate(mesh1)
kdx = k * fd.dot(d, x)
inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
g = -inc
u1 = solve(mesh1, k, a0, a1, b0, b1, g, annular=annular)
plot_field(u1, a0, a1, b0, b1)
plot_far_field_vol(k, u1, R0, R1)

n = 20
theta = 2 * np.pi / n * np.arange(n)
ratio = []
for t in list(theta):
    x_hat = fd.Constant((np.cos(t), np.sin(t)))
    u_inf0_re, u_inf0_im = far_field_vol(k, u0, x_hat, R0, R1)
    u_inf1_re, u_inf1_im = far_field_vol(k, u1, x_hat, R0, R1)
    phi = k * h.dot(d_np - np.array([np.cos(t), np.sin(t)]))
    
    u_inf0_re_ = np.cos(phi) * u_inf0_re - np.sin(phi) * u_inf0_im
    u_inf0_im_ = np.cos(phi) * u_inf0_im + np.sin(phi) * u_inf0_re
    ratio_re = u_inf0_re_ / u_inf1_re
    ratio_im = u_inf0_im_ / u_inf1_im
    ratio.append((ratio_re, ratio_im))
theta = np.append(theta, 0)
ratio = np.array(ratio + [ratio[0]])

fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={'projection': 'polar'},
                               constrained_layout=True)
ax1.plot(theta, ratio[:, 0])
ax1.set_title("Real part")
ax1.set_rlabel_position(90)
ax1.grid(True)
ax2.plot(theta, ratio[:, 1])
ax2.set_title("Imaginary part")
ax2.set_rlabel_position(90)
ax2.grid(True)
fig.suptitle("Translation property")

## Kite-shaped obstacle

In [ ]:
# shape of obstacle
shape = "kite"
# set up PML
a0 = b0 = 2.75
a1 = b1 = 3.0
h0 = (a1 - a0) / N
# set up cut-off function for far field
R0 = 2.3
R1 = 2.7
if R0 is not None and R1 is not None:
    annular = True
else:
    annular = False

# generate mesh
cached_mesh = False
if cached_mesh:
    mesh = fd.Mesh(shape + str(0) + ".msh")
else:
    mesh = generate_mesh(a0, a1, b0, b1, shape, N, R0=R0, R1=R1)

In [ ]:
# visualize mesh
plot_mesh(mesh)

In [ ]:
# wavenumber of incident wave
k = 5
# direction of incident wave
d = fd.Constant([1., 0.])

# Dirichlet boundary condition for sound-soft obstacle
x = fd.SpatialCoordinate(mesh)
kdx = k * fd.dot(d, x)
inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
g = -inc

# compute approximate solution
uh = solve(mesh, k, a0, a1, b0, b1, g, annular=annular)

In [ ]:
# visualize scattered field
plot_field(uh, a0, a1, b0, b1)

In [ ]:
# visualize total field
u = fd.interpolate(uh + inc, uh.function_space())
plot_field(u, a0, a1, b0, b1)

In [ ]:
# plot far field pattern evaluated by boundary-based formula
# use boundary of obstacle
plot_far_field(k, uh)

In [ ]:
# plot far field pattern evaluated by boundary-based formula
# use arbitrary closed surface enclosing obstacle
plot_far_field(k, uh, boundary=4)

In [ ]:
# plot far field pattern evaluated by volume-based formula
plot_far_field_vol(k, uh, R0, R1)

In [ ]:
# verify reciprocity relation [1, Theorem 3.23]
cached_mesh = False
if cached_mesh:
    mesh = fd.Mesh(shape + str(3) + ".msh")
else:
    mesh = generate_mesh(a0, a1, b0, b1, shape, N, level=3, R0=R0, R1=R1)
k = 5
d = fd.Constant([1., 0.])
x = fd.SpatialCoordinate(mesh)
kdx = k * fd.dot(d, x)
inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
g = -inc
u1 = solve(mesh, k, a0, a1, b0, b1, g, annular=annular)

n = 20
theta = 2 * np.pi / n * np.arange(n)
ratio = []
for t in list(theta):
    x_hat = fd.Constant((np.cos(t), np.sin(t)))
    u_inf1_re, u_inf1_im = far_field_vol(k, u1, x_hat, R0, R1)
    
    x = fd.SpatialCoordinate(mesh)
    kdx = k * fd.dot(-x_hat, x)
    inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
    g = -inc
    u2 = solve(mesh, k, a0, a1, b0, b1, g, annular=annular)
    u_inf2_re, u_inf2_im = far_field_vol(k, u2, -d, R0, R1)

    ratio.append((u_inf2_re / u_inf1_re, u_inf2_im / u_inf1_im))
theta = np.append(theta, 0)
ratio = np.array(ratio + [ratio[0]])

fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={'projection': 'polar'},
                               constrained_layout=True)
ax1.plot(theta, ratio[:, 0])
ax1.set_title("Real part")
ax1.set_rlabel_position(90)
ax1.grid(True)
ax2.plot(theta, ratio[:, 1])
ax2.set_title("Imaginary part")
ax2.set_rlabel_position(90)
ax2.grid(True)
fig.suptitle("Reciprocity relation")

In [ ]:
# set up convergence test for far field
max_level = 4
levels = np.arange(max_level)
hs = h0 / 2**levels

In [ ]:
# create mesh hierarchy by uniform refinement
mesh_hierarchy = []
if cached_mesh:
    for level in levels:
        mesh_hierarchy.append(fd.Mesh(shape + str(level) + ".msh"))
else:
    for level in levels:
        mesh_hierarchy.append(generate_mesh(a0, a1, b0, b1, shape, N, level, R0=R0, R1=R1))

Reference value from [1, Table 3.1]

- $k = 1$

    $Re\{u_\infty(d)\} = −1.62745750, Im\{u_\infty(d)\} = 0.60222591$

    $Re\{u_\infty(-d)\} = 1.39694488, Im\{u_\infty(-d)\} = 0.09499635$

- $k = 5$

    $Re\{u_\infty(d)\} = -2.47554380, Im\{u_\infty(d)\} = 1.68747937$

    $Re\{u_\infty(-d)\} = -0.19945787, Im\{u_\infty(-d)\} = 0.06015893$


In [ ]:
k = 1

# set reference value
if k == 1:
    u_inf_ref = np.array([-1.62745750, 0.60222591, 1.39694488, 0.09499635])
if k == 5:
    u_inf_ref = np.array([-2.47554380, 1.68747937, -0.19945787, 0.06015893])

In [ ]:
# test of boundary-based formula
# use boundary of obstacle
errors = []
for level in levels:
    m = mesh_hierarchy[level]
    x = fd.SpatialCoordinate(m)
    kdx = k * fd.dot(d, x)
    inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
    g = -inc
    
    uh = solve(m, k, a0, a1, b0, b1, g, annular=annular)

    u_inf = np.empty(4)
    u_inf[0], u_inf[1] = far_field(k, uh, d)
    u_inf[2], u_inf[3] = far_field(k, uh, -d)
    rel_err = np.abs((u_inf - u_inf_ref) / u_inf_ref)
    print(f"refinement level {level}, relative error {rel_err}")
    errors.append(rel_err)
errors = np.array(errors)
plt.loglog(hs, errors[:, 0], "-o",
           label=r"Relative error of $Re\{u_\infty(d)\}$")
plt.loglog(hs, errors[:, 1], "-o",
           label=r"Relative error of $Im\{u_\infty(d)\}$")
plt.loglog(hs, errors[:, 2], "-o",
           label=r"Relative error of $Re\{u_\infty(-d)\}$")
plt.loglog(hs, errors[:, 3], "-o",
           label=r"Relative error of $Im\{u_\infty(-d)\}$")

plt.loglog(hs, hs, "k", label=r"Order $h$")
plt.legend()
plt.xlabel(r"Mesh width $h$")
plt.ylabel("Relative error")
plt.title("Mesh-boundary-based formula")
plt.tight_layout()

In [ ]:
# test of boundary-based formula
# use arbitrary closed surface enclosing obstacle
errors = []
for level in levels:
    m = mesh_hierarchy[level]
    x = fd.SpatialCoordinate(m)
    kdx = k * fd.dot(d, x)
    inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
    g = -inc
    
    uh = solve(m, k, a0, a1, b0, b1, g, annular=annular)

    u_inf = np.empty(4)
    u_inf[0], u_inf[1] = far_field(k, uh, d, boundary=4)
    u_inf[2], u_inf[3] = far_field(k, uh, -d, boundary=4)
    rel_err = np.abs((u_inf - u_inf_ref) / u_inf_ref)
    print(f"refinement level {level}, relative error {rel_err}")
    errors.append(rel_err)
errors = np.array(errors)
plt.loglog(hs, errors[:, 0], "-o",
           label=r"Relative error of $Re\{u_\infty(d)\}$")
plt.loglog(hs, errors[:, 1], "-o",
           label=r"Relative error of $Im\{u_\infty(d)\}$")
plt.loglog(hs, errors[:, 2], "-o",
           label=r"Relative error of $Re\{u_\infty(-d)\}$")
plt.loglog(hs, errors[:, 3], "-o",
           label=r"Relative error of $Im\{u_\infty(-d)\}$")

plt.loglog(hs, hs, "k", label=r"Order $h$")
plt.legend()
plt.xlabel(r"Mesh width $h$")
plt.ylabel("Relative error")
plt.title("Arbitrary-boundary-based formula")
plt.tight_layout()

In [ ]:
# test of volume-based formula
errors = []
for level in levels:
    m = mesh_hierarchy[level]
    x = fd.SpatialCoordinate(m)
    kdx = k * fd.dot(d, x)
    inc = fd.as_vector((fd.cos(kdx), fd.sin(kdx)))
    g = -inc
    
    uh = solve(m, k, a0, a1, b0, b1, g, annular=annular)

    u_inf = np.empty(4)
    u_inf[0], u_inf[1] = far_field_vol(k, uh, d, R0, R1)
    u_inf[2], u_inf[3] = far_field_vol(k, uh, -d, R0, R1)
    rel_err = np.abs((u_inf - u_inf_ref) / u_inf_ref)
    print(f"refinement level {level}, relative error {rel_err}")
    errors.append(rel_err)
errors = np.array(errors)
plt.loglog(hs, errors[:, 0], "-o",
           label=r"Relative error of $Re\{u_\infty(d)\}$")
plt.loglog(hs, errors[:, 1], "-o",
           label=r"Relative error of $Im\{u_\infty(d)\}$")
plt.loglog(hs, errors[:, 2], "-o",
           label=r"Relative error of $Re\{u_\infty(-d)\}$")
plt.loglog(hs, errors[:, 3], "-o",
           label=r"Relative error of $Im\{u_\infty(-d)\}$")

plt.loglog(hs, hs**2, "k", label=r"Order $h^2$")
plt.legend()
plt.xlabel(r"Mesh width $h$")
plt.ylabel("Relative error")
plt.title("Volume-based formula")
plt.tight_layout()

## Reference

[1] Colton, D., Kress, R., 2019. Inverse Acoustic and Electromagnetic Scattering Theory, Applied Mathematical Sciences. Springer International Publishing, Cham. https://doi.org/10.1007/978-3-030-30351-8